In [ ]:
# Install dependencies
%pip install -e '.[kafka-python]'

# Import the reliable topic creation functions
from diaspora_event_sdk.sdk.kafka_client import (
    reliable_topic_creation,
    reliable_topic_deletion,
    KafkaProducer,
    KafkaConsumer,
)
from diaspora_event_sdk import Client as GlobusClient
import json
from datetime import datetime
import os

# Initialize client for recreate_topic_v3
os.environ["DIASPORA_SDK_ENVIRONMENT"] = "local"

c = GlobusClient()


In [ ]:
# Step 1: Create a topic
topic_name = reliable_topic_creation()
print(f"Created topic: {topic_name}")

In [ ]:
# # Step 2: Recreate topic to truncate existing messages
# namespace, topic = topic_name.split(".", 1)
# print(f"\nRecreating topic to truncate messages: {topic_name}")
# result = c.recreate_topic_v3(namespace, topic)
# print(f"Recreate result: {result.get('status', 'N/A')}, {result.get('message', 'N/A')}")

In [ ]:
# Step 3: Produce three messages with timestamps
print(f"\nProducing messages to topic: {topic_name}")
producer = KafkaProducer(topic_name)
for i in range(3):
    message = {
        "message_id": i + 1,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "content": f"Message {i + 1} from reliable_topic_creation",
    }
    future = producer.send(topic_name, message)
    result = future.get(timeout=10)
    print(f"Produced message {i + 1}: offset={result.offset}, timestamp={message['timestamp']}")
producer.close()

In [ ]:
# Step 4: Consume the three messages
print(f"\nConsuming messages from topic: {topic_name}")
consumer = KafkaConsumer(topic_name, auto_offset_reset="earliest")
messages = consumer.poll(timeout_ms=10000)
consumed_count = 0
for tp, msgs in messages.items():
    for message in msgs:
        data = json.loads(message.value.decode("utf-8"))
        consumed_count += 1
        print(f"Consumed message {consumed_count}: {data}")
consumer.close()
print(f"\nTotal messages consumed: {consumed_count}")


In [ ]:
# Step 5: Delete the topic
result = reliable_topic_deletion(topic_name)
print(f"\nDeletion result: {result}")
